<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 1. Inference

In [11]:
import os
import sys
import timeit
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf

from deepcell.applications import NuclearSegmentation
from deepcell.applications.cell_tracking import CONFIGS
from deepcell_tracking.tracking import CellTracker as _CellTracker
from deepcell_tracking.tracking import linear_sum_assignment
from deepcell.datasets import DynamicNuclearNetTracking
from deepcell.utils import fetch_data, extract_archive

sys.path.append('..')
import utils

In [2]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

In [14]:
# Add timers to CellTracker
class CellTracker(_CellTracker):

    def __init__(self, X, y, model, neighborhood_encoder, **kwargs):
        # Add timers
        self.timer_ne = 0
        self.timer_inf = 0
        self.timer_lap = 0

        super().__init__(X, y, model, neighborhood_encoder=neighborhood_encoder, **kwargs)

    def _get_neighborhood_embeddings(self, **kwargs):
        tic = timeit.default_timer()
        embeddings = super()._get_neighborhood_embeddings(**kwargs)
        toc = timeit.default_timer()

        self.timer_ne += toc - tic

        return embeddings

    def _track_frame(self, frame):
        tic = timeit.default_timer()
        cost_matrix, predictions = self._get_cost_matrix(frame)
        toc = timeit.default_timer()

        self.timer_inf += toc - tic

        tic = timeit.default_timer()
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        assignments = np.stack([row_ind, col_ind], axis=1)
        self._update_tracks(assignments, frame, predictions)
        toc = timeit.default_timer()

        self.timer_lap += toc - tic


In [6]:
# Load test data
dnn = DynamicNuclearNetTracking(version='1.1')
X, y, lineages = dnn.load_data(split='test')
data = {
    'X': X,
    'y': y,
    'lineages': lineages
}

INFO:root:Checking for cached data
INFO:root:Checking DynamicNuclearNet-tracking-v1_1.zip against provided file_hash...
INFO:root:DynamicNuclearNet-tracking-v1_1.zip with hash 4090ccd3b76c27f001f798c8187c5f3e already available.
INFO:root:Extracting /home/msschwar/.deepcell/datasets/DynamicNuclearNet-tracking-v1_1.zip
INFO:root:Successfully extracted /home/msschwar/.deepcell/datasets/DynamicNuclearNet-tracking-v1_1.zip into /home/msschwar/.deepcell/datasets


In [7]:
app_seg = NuclearSegmentation.from_version(version='1.1')

INFO:root:Checking for cached data
INFO:root:Checking NuclearSegmentation-8.tar.gz against provided file_hash...
INFO:root:NuclearSegmentation-8.tar.gz with hash 507be21f0e34e59adae689f58cc03ccb already available.
INFO:root:Extracting /home/msschwar/.deepcell/models/NuclearSegmentation-8.tar.gz
INFO:root:Successfully extracted /home/msschwar/.deepcell/models/NuclearSegmentation-8.tar.gz into /home/msschwar/.deepcell/models
2024-07-01 14:43:57.789271: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 14:43:57.907669: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46873 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus

In [12]:
# Retrieve tracking models
config = CONFIGS['1.1']

cache_subdir = "models"
model_dir = Path.home() / ".deepcell" / "models"

# Load encoder
archive_path = fetch_data(
    asset_key=config.encoder_key,
    cache_subdir=cache_subdir,
    file_hash=config.encoder_hash
)
extract_archive(archive_path, model_dir)
model_path = model_dir / 'NuclearTrackingNE'
neighborhood_encoder = tf.keras.models.load_model(model_path)

# Load inference
archive_path = fetch_data(
    asset_key=config.inf_key,
    cache_subdir=cache_subdir,
    file_hash=config.inf_hash
)
extract_archive(archive_path, model_dir)
model_path = model_dir / 'NuclearTrackingInf'
inference = tf.keras.models.load_model(model_path)

INFO:root:Checking for cached data
INFO:root:Checking NuclearTrackingNE-8.tar.gz against provided file_hash...
INFO:root:NuclearTrackingNE-8.tar.gz with hash 79188d7ae32b40b5bd0ad0f2ac2b53c4 already available.
INFO:root:Extracting /home/msschwar/.deepcell/models/NuclearTrackingNE-8.tar.gz
INFO:root:Successfully extracted /home/msschwar/.deepcell/models/NuclearTrackingNE-8.tar.gz into /home/msschwar/.deepcell/models


INFO:root:Checking for cached data
INFO:root:Checking NuclearTrackingInf-8.tar.gz against provided file_hash...
INFO:root:NuclearTrackingInf-8.tar.gz with hash ec07d8c0770453e738f8699ceede78e7 already available.
INFO:root:Extracting /home/msschwar/.deepcell/models/NuclearTrackingInf-8.tar.gz
INFO:root:Successfully extracted /home/msschwar/.deepcell/models/NuclearTrackingInf-8.tar.gz into /home/msschwar/.deepcell/models


In [15]:
timers = []
for it in range(3):
    for batch_no in range(len(data['lineages'])):
        print(batch_no)
        tm = {'batch': batch_no, 'iteration': it}

        # Pull out relevant data for this batch
        X = data['X'][batch_no]
        y = data['y'][batch_no]
        lineage = data['lineages'][batch_no]

        # Correct discontiguous tracks, which are not allowed by CTC
        y, lineage = utils.convert_to_contiguous(y, lineage)

        # Determine position of zero padding for removal
        slc = utils.find_zero_padding(X)
        X = X[slc]
        y = y[slc]

        # Determine which frames are zero padding
        frames = np.sum(y, axis=(1,2)) # True if image not blank
        good_frames = np.where(frames)[0]
        X = X[:len(good_frames)]
        y = y[:len(good_frames)]

        # Generate tracks on predicted segmentations
        tic = timeit.default_timer()
        y_pred = app_seg.predict(X)
        tm['seg'] = timeit.default_timer() - tic

        tracker = CellTracker(
            X,
            y_pred,
            inference,
            neighborhood_encoder=neighborhood_encoder,
            distance_threshold=config.distance_threshold,
            appearance_dim=config.appearance_dim,
            birth=config.birth,
            death=config.death,
            division=config.division,
            track_length=config.track_length,
            embedding_axis=0,
            crop_mode=config.crop_mode,
            norm=config.norm,
        )
        tracker.track_cells()
        tm['ne'] = tracker.timer_ne
        tm['inf'] = tracker.timer_inf
        tm['lap'] = tracker.timer_lap

        tm['nframes'] = X.shape[0]
        tm['nobjects'] = sum([len(np.unique(y[t])) - 1 for t in range(y.shape[0])])

        timers.append(tm)

INFO:root:Converting image dtype to float


0


2024-07-01 14:54:45.707145: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
INFO:CellTracker:Created new track for cell 187.
INFO:CellTracker:Created new track for cell 188.
INFO:CellTracker:Detected division! Cell 188 is daughter of cell 118.


INFO:CellTracker:Created new track for cell 190.
INFO:CellTracker:Detected division! Cell 190 is daughter of cell 54.
INFO:CellTracker:Created new track for cell 191.
INFO:CellTracker:Detected division! Cell 191 is daughter of cell 63.


INFO:CellTracker:Created new track for cell 194.
INFO:CellTracker:Created new track for cell 195.
INFO:CellTracker:Detected division! Cell 195 is daughter of cell 27.
INFO:CellTracker:Created new track for cell 196.
INFO:CellTracker:Detected division! Cell 196 is daughter of cell 62.
INFO:CellTracker:Created new track for cell 197.
INFO:CellTracker:Created new track for cell 200.
INFO:CellTracker:Detected division! Cell 200 is daughter of cell 103.
INFO:CellTracker:Created new track for cell 201.
INFO:CellTracker:Detected division! Cell 201 is daughter of cell 103.
INFO:CellTracker:Created new track for cell 202.
INFO:CellTracker:Created new track for cell 203.
INFO:CellTracker:Detected division! Cell 203 is daughter of cell 7.
INFO:CellTracker:Created new track for cell 204.
INFO:CellTracker:Created new track for cell 206.
INFO:CellTracker:Created new track for cell 207.
INFO:CellTracker:Created new track for cell 208.
INFO:CellTracker:Detected division! Cell 208 is daughter of cell 1

...

INFO:CellTracker:Created new track for cell 77.
INFO:CellTracker:Created new track for cell 78.
INFO:CellTracker:Created new track for cell 79.
INFO:CellTracker:Created new track for cell 80.
INFO:CellTracker:Detected division! Cell 80 is daughter of cell 43.
INFO:CellTracker:Created new track for cell 82.
INFO:CellTracker:Detected division! Cell 82 is daughter of cell 38.
INFO:CellTracker:Created new track for cell 83.
INFO:CellTracker:Detected division! Cell 83 is daughter of cell 38.
INFO:CellTracker:Created new track for cell 84.
INFO:CellTracker:Created new track for cell 85.
INFO:CellTracker:Created new track for cell 86.
INFO:CellTracker:Created new track for cell 87.
INFO:CellTracker:Created new track for cell 88.
INFO:CellTracker:Detected division! Cell 88 is daughter of cell 49.
INFO:CellTracker:Created new track for cell 89.
INFO:CellTracker:Detected division! Cell 89 is daughter of cell 49.
INFO:CellTracker:Created new track for cell 90.
INFO:CellTracker:Detected division! 

10


INFO:CellTracker:Created new track for cell 122.
INFO:CellTracker:Detected division! Cell 122 is daughter of cell 6.
INFO:CellTracker:Created new track for cell 124.
INFO:CellTracker:Created new track for cell 125.
INFO:CellTracker:Created new track for cell 126.
INFO:CellTracker:Created new track for cell 127.
INFO:CellTracker:Created new track for cell 128.
INFO:CellTracker:Created new track for cell 129.
INFO:CellTracker:Created new track for cell 130.
INFO:CellTracker:Detected division! Cell 130 is daughter of cell 116.
INFO:CellTracker:Created new track for cell 131.
INFO:CellTracker:Detected division! Cell 131 is daughter of cell 116.
INFO:CellTracker:Created new track for cell 132.
INFO:CellTracker:Detected division! Cell 132 is daughter of cell 74.
INFO:CellTracker:Created new track for cell 133.
INFO:CellTracker:Created new track for cell 135.
INFO:CellTracker:Detected division! Cell 135 is daughter of cell 128.
INFO:CellTracker:Created new track for cell 136.
INFO:CellTracker

11


INFO:CellTracker:Created new track for cell 180.
INFO:CellTracker:Created new track for cell 181.
INFO:CellTracker:Created new track for cell 182.
INFO:CellTracker:Created new track for cell 183.
INFO:CellTracker:Created new track for cell 184.
INFO:CellTracker:Created new track for cell 185.
INFO:CellTracker:Created new track for cell 186.
INFO:CellTracker:Created new track for cell 187.
INFO:CellTracker:Created new track for cell 188.
INFO:CellTracker:Detected division! Cell 188 is daughter of cell 84.
INFO:CellTracker:Created new track for cell 189.
INFO:CellTracker:Detected division! Cell 189 is daughter of cell 136.
INFO:CellTracker:Created new track for cell 192.
INFO:CellTracker:Created new track for cell 193.
INFO:CellTracker:Detected division! Cell 193 is daughter of cell 12.
INFO:CellTracker:Created new track for cell 194.
INFO:CellTracker:Created new track for cell 195.
INFO:CellTracker:Created new track for cell 196.
INFO:CellTracker:Detected division! Cell 196 is daughter 

In [16]:
df = pd.DataFrame(timers)
df.to_csv('speed.csv')